In [128]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import pandas_datareader.data as web
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from matplotlib import pyplot
import scipy.stats
from pytrends import dailydata
from statsmodels.tsa.ar_model import AR
from yahoofinancials import YahooFinancials
%matplotlib inline
import pytrends
from sklearn.decomposition import PCA

In [114]:
from pytrends.request import TrendReq
pytrend = TrendReq(tz = 360)
 

In [ ]:
def grangerTest(exog, endog):
    MAX_LAG = 30
    ARaic = AR(exog.tolist()).fit(maxlag=MAX_LAG, ic="aic")
    ARbic = AR(exog.tolist()).fit(maxlag=MAX_LAG, ic="bic")
    # select the fewer number of parameters between both criteria.
    numExog = len(ARaic.params) if len(ARaic.params) < len(ARbic.params) else len(ARbic.params)

    print ("Optimal number of lags for exog data is " + str(numExog))

    ARaic = AR(endog.tolist()).fit(maxlag=MAX_LAG, ic="aic")
    ARbic = AR(endog.tolist()).fit(maxlag=MAX_LAG, ic="bic")
    # select the fewer number of parameters between both criteria.
    numEndog = len(ARaic.params) if len(ARaic.params) < len(ARbic.params) else len(ARbic.params)

    print ("Optimal number of lags for endog data is " + str(numEndog))

    # now that I know the optimal number of parameters, I can call the
    # granger causality function of statsmodels.
    data = pd.concat([endog, exog], axis=1)
    print ("\nGranger causality results of indep onto dep")
    results = grangercausalitytests(data, maxlag=numEndog)

    data = pd.concat([exog, endog], axis=1)
    print ("\nGranger causality results of dep onto indep")
    results = grangercausalitytests(data, maxlag=numExog)
    regr = results[2][1]
    print (regr[0].params)
    print (regr[1].params)
    print (regr[1].pvalues)

## Downlaod S&P

In [192]:
#SnP_daily = web.get_data_yahoo('^GSPC','01/01/2005','05/01/2021',interval='d')
yahoo_financials = YahooFinancials(['^GSPC']).get_historical_price_data('2019-01-01','2021-05-31','daily')
SnP_daily =pd.DataFrame(yahoo_financials["^GSPC"]["prices"])
SnP_daily.index = SnP_daily['formatted_date']
SnP_daily['LogReturn'] = np.log(SnP_daily['adjclose']/SnP_daily['adjclose'].shift(1))
SnP_daily
#yahoo_financials["^GSPC"]["prices"]

,date,high,low,open,close,volume,adjclose,formatted_date,LogReturn
formatted_date,,,,,,,,,
2019-01-02,1546439400,2519.489990,2467.469971,2476.959961,2510.030029,3733160000,2510.030029,2019-01-02,NaN
2019-01-03,1546525800,2493.139893,2443.959961,2491.919922,2447.889893,3822860000,2447.889893,2019-01-03,-0.025068
2019-01-04,1546612200,2538.070068,2474.330078,2474.330078,2531.939941,4213410000,2531.939941,2019-01-04,0.033759
2019-01-07,1546871400,2566.159912,2524.560059,2535.610107,2549.689941,4104710000,2549.689941,2019-01-07,0.006986
2019-01-08,1546957800,2579.820068,2547.560059,2568.110107,2574.409912,4083030000,2574.409912,2019-01-08,0.009649
...,...,...,...,...,...,...,...,...,...
2021-05-24,1621863000,4209.520020,4170.160156,4170.160156,4197.049805,2947400000,4197.049805,2021-05-24,0.009862
2021-05-25,1621949400,4213.419922,4182.520020,4205.939941,4188.129883,3420870000,4188.129883,2021-05-25,-0.002128
2021-05-26,1622035800,4202.609863,4184.109863,4191.589844,4195.990234,3674490000,4195.990234,2021-05-26,0.001875


In [149]:
SnP_daily['LogReturn'] = np.log(SnP_daily['adjclose']/SnP_daily['adjclose'].shift(1))

In [ ]:
Granger_data_daily = pd.merge(SnP_daily,df, how= 'inner', left_index=True, right_index=True)

In [ ]:

    
grangerTest(Granger_data_daily['S&P_unscaled'][1:], Granger_data_daily['LogReturn'][1:])

In [ ]:
SnP_daily = web.get_data_yahoo('^GSPC','01/01/2005','05/01/2021',interval='d')
SnP_daily['LogReturn'] = np.log(SnP_daily['Adj Close']/SnP_daily['Adj Close'].shift(1))


In [ ]:

descriptors = None

## Download descriptors

In [112]:
key_words =[
#'Financial Crisis'
#'Depression'
#,'Bailout'
#,'Mortgage Crisis'
#,'Wall Street'
#,'Oil'
#'Stock Market'
#'Subprime'
#,'Credit Crisis'
#,'Housing Crisis'
#,'loan'
#,'core inflation'
#,'debt'
#,'Brent'
#,'WTI'
#,'metals'
#,'inflation'
#,'economics'
#,'stocks'
#,'portfolio'
#,'S&P 500'

#'gold'
#'silver'
'S&P 500'
]

In [116]:
for word in key_words:
    df = dailydata.get_daily_data(word, 2005, 1, 2021, 5, geo = 'US')
    all_descriptors = pd.merge(all_descriptors,df[word+'_unscaled'], how= 'inner', left_index=True, right_index=True)

S&P 500:2005-01-01 2005-01-31
S&P 500:2005-02-01 2005-02-28
S&P 500:2005-03-01 2005-03-31
S&P 500:2005-04-01 2005-04-30
S&P 500:2005-05-01 2005-05-31
S&P 500:2005-06-01 2005-06-30
S&P 500:2005-07-01 2005-07-31
S&P 500:2005-08-01 2005-08-31
S&P 500:2005-09-01 2005-09-30
S&P 500:2005-10-01 2005-10-31
S&P 500:2005-11-01 2005-11-30
S&P 500:2005-12-01 2005-12-31
S&P 500:2006-01-01 2006-01-31
S&P 500:2006-02-01 2006-02-28
S&P 500:2006-03-01 2006-03-31
S&P 500:2006-04-01 2006-04-30
S&P 500:2006-05-01 2006-05-31
S&P 500:2006-06-01 2006-06-30
S&P 500:2006-07-01 2006-07-31
S&P 500:2006-08-01 2006-08-31
S&P 500:2006-09-01 2006-09-30
S&P 500:2006-10-01 2006-10-31
S&P 500:2006-11-01 2006-11-30
S&P 500:2006-12-01 2006-12-31
S&P 500:2007-01-01 2007-01-31
S&P 500:2007-02-01 2007-02-28
S&P 500:2007-03-01 2007-03-31
S&P 500:2007-04-01 2007-04-30
S&P 500:2007-05-01 2007-05-31
S&P 500:2007-06-01 2007-06-30
S&P 500:2007-07-01 2007-07-31
S&P 500:2007-08-01 2007-08-31
S&P 500:2007-09-01 2007-09-30
S&P 500:20

In [100]:
covid_words = [#'covid-19',
               #'pandemic',
            #'covid vaccine',
            'social distancing']
for word in covid_words:
    df = dailydata.get_daily_data(word, 2020, 3, 2021, 5, geo = 'US')
    descriptors_covid = pd.merge(descriptors_covid,df[word+'_unscaled'], how= 'inner', left_index=True, right_index=True)

social distancing:2020-03-01 2020-03-31
social distancing:2020-04-01 2020-04-30
social distancing:2020-05-01 2020-05-31
social distancing:2020-06-01 2020-06-30
social distancing:2020-07-01 2020-07-31
social distancing:2020-08-01 2020-08-31
social distancing:2020-09-01 2020-09-30
social distancing:2020-10-01 2020-10-31
social distancing:2020-11-01 2020-11-30
social distancing:2020-12-01 2020-12-31
social distancing:2021-01-01 2021-01-31
social distancing:2021-02-01 2021-02-28
social distancing:2021-03-01 2021-03-31
social distancing:2021-04-01 2021-04-30
social distancing:2021-05-01 2021-05-31


In [101]:
descriptors_covid.head()

,covid-19_unscaled,pandemic_unscaled,covid vaccine_unscaled,social distancing_unscaled
date,,,,
2020-03-01,5,8,5,0
2020-03-02,9,10,8,0
2020-03-03,9,9,8,0
2020-03-04,10,7,10,0
2020-03-05,14,7,9,1


In [ ]:
descriptors_2_2 = pd.read_csv('Google_Trends_variables.tsv', sep = '\t')
descriptors_2_2.index  = descriptors_2_2["Date"]
del descriptors_2_2["Date"]
del descriptors_2_2["LogReturn"]
descriptors_2_2 = pd.merge(descriptors_2_2,descriptors_2[['gold_unscaled','silver_unscaled']], how= 'inner', left_index=True, right_index=True)
descriptors_2_2.head()

In [ ]:
descriptors_1 = descriptors 
del descriptors_1["Credit Crisis_unscaled"]
print(descriptors_2_2.columns)
print(descriptors_1.columns)
print(descriptors_2_2.columns == descriptors_1.columns)

In [ ]:
df = dailydata.get_daily_data('covid-19', 2020, 3, 2021, 5, geo = 'US')
descriptors_covid = df

In [ ]:
all_descriptors = pd.concat([descriptors_1, descriptors_2_2])

In [121]:
DickeyFuller_output = pd.DataFrame()
DickeyFuller_output["Predictors"] = all_descriptors.columns
for i in range(len(DickeyFuller_output["Predictors"])):
    DickeyFuller_output["Predictors"][i] = DickeyFuller_output["Predictors"][i].replace("_unscaled",'')
DickeyFuller_output["Dickey-Fuller p-values"] = None
for j in range(len(DickeyFuller_output["Predictors"])):
    dftest = adfuller(all_descriptors[all_descriptors.columns[j]].dropna(), autolag='AIC')
    DickeyFuller_output["Dickey-Fuller p-values"][j] = round(dftest[1],4) 
DickeyFuller_output

,Predictors,Dickey-Fuller p-values
0,Financial Crisis,0
1,Depression,0
2,Bailout,0
3,Mortgage Crisis,0
4,Wall Street,0
5,Oil,0
6,Stock Market,0
7,Subprime,0
8,loan,0
9,core inflation,0


In [102]:

DickeyFuller_output_covid = pd.DataFrame()
DickeyFuller_output_covid["Predictors"] = descriptors_covid.columns
for i in range(len(DickeyFuller_output_covid["Predictors"])):
    DickeyFuller_output_covid["Predictors"][i] = DickeyFuller_output_covid["Predictors"][i].replace("_unscaled",'')
DickeyFuller_output_covid["Dickey-Fuller p-values"] = None
for j in range(len(DickeyFuller_output_covid["Predictors"])):
    dftest = adfuller(descriptors_covid[descriptors_covid.columns[j]].dropna(), autolag='AIC')
    DickeyFuller_output_covid["Dickey-Fuller p-values"][j] = round(dftest[1],4) 
DickeyFuller_output_covid

,Predictors,Dickey-Fuller p-values
0,covid-19,0.0016
1,pandemic,0.0132
2,covid vaccine,0.0006
3,social distancing,0.0007


In [103]:
print(DickeyFuller_output_covid.to_latex())

\begin{tabular}{lll}
\toprule
{} &         Predictors & Dickey-Fuller p-values \\
\midrule
0 &           covid-19 &                 0.0016 \\
1 &           pandemic &                 0.0132 \\
2 &      covid vaccine &                 0.0006 \\
3 &  social distancing &                 0.0007 \\
\bottomrule
\end{tabular}



In [81]:
print(DickeyFuller_output.to_latex())

\begin{tabular}{lll}
\toprule
{} &        Predictors & Dickey-Fuller p-values \\
\midrule
0  &  Financial Crisis &                      0 \\
1  &        Depression &                      0 \\
2  &           Bailout &                      0 \\
3  &   Mortgage Crisis &                      0 \\
4  &       Wall Street &                      0 \\
5  &               Oil &                      0 \\
6  &      Stock Market &                      0 \\
7  &          Subprime &                      0 \\
8  &              loan &                      0 \\
9  &    core inflation &                      0 \\
10 &              debt &                      0 \\
11 &             Brent &                      0 \\
12 &               WTI &                 0.0003 \\
13 &            metals &                      0 \\
14 &         inflation &                      0 \\
15 &         economics &                      0 \\
16 &            stocks &                      0 \\
17 &              gold &                   

In [118]:
all_transposed = all_descriptors.describe().transpose()

    
covid_transposed = descriptors_covid.describe().transpose()

print(pd.concat([all_transposed,covid_transposed]).to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &   count &       mean &        std &  min &   25\% &   50\% &   75\% &    max \\
\midrule
Financial Crisis\_unscaled  &  4433.0 &  41.991879 &  27.036927 &  0.0 &  22.0 &  41.0 &  60.0 &  100.0 \\
Depression\_unscaled        &  4433.0 &  78.081209 &  14.441535 &  0.0 &  70.0 &  80.0 &  89.0 &  100.0 \\
Bailout\_unscaled           &  4433.0 &  39.660726 &  26.972700 &  0.0 &  19.0 &  38.0 &  57.0 &  100.0 \\
Mortgage Crisis\_unscaled   &  4433.0 &  25.102865 &  29.454552 &  0.0 &   0.0 &  20.0 &  43.0 &  100.0 \\
Wall Street\_unscaled       &  4433.0 &  68.174148 &  18.506326 &  0.0 &  56.0 &  70.0 &  81.0 &  100.0 \\
Oil\_unscaled               &  4433.0 &  85.648545 &  10.096985 &  0.0 &  82.0 &  87.0 &  92.0 &  100.0 \\
Stock Market\_unscaled      &  4433.0 &  58.283781 &  22.452000 &  0.0 &  42.0 &  60.0 &  75.0 &  100.0 \\
Subprime\_unscaled          &  4433.0 &  40.074216 &  26.412611 &  0.0 &  20.0 &  37.0 &  57.0 &  100.0 \\
loan\_unscaled

In [200]:
merged = pd.merge(all_descriptors,SnP_daily['LogReturn'], how= 'inner', left_index=True, right_index=True)
corr = merged[["S&P 500_unscaled","stocks_unscaled","Stock Market_unscaled","Wall Street_unscaled","Oil_unscaled"]].corr()
corr.style.background_gradient(cmap='coolwarm')

,S&P 500_unscaled,stocks_unscaled,Stock Market_unscaled,Wall Street_unscaled,Oil_unscaled
S&P 500_unscaled,1.000000,0.473830,0.605714,0.111953,0.067516
stocks_unscaled,0.473830,1.000000,0.657574,0.331074,0.192012
Stock Market_unscaled,0.605714,0.657574,1.000000,0.242708,0.101984
Wall Street_unscaled,0.111953,0.331074,0.242708,1.000000,-0.198794
Oil_unscaled,0.067516,0.192012,0.101984,-0.198794,1.000000


In [171]:
corr = merged2.corr()
corr.style.background_gradient(cmap='coolwarm')

,covid-19_unscaled,pandemic_unscaled,covid vaccine_unscaled,social distancing_unscaled,LogReturn
covid-19_unscaled,1.000000,0.107463,0.448219,0.095765,-0.043874
pandemic_unscaled,0.107463,1.000000,0.066459,0.224079,0.017769
covid vaccine_unscaled,0.448219,0.066459,1.000000,-0.034366,-0.055334
social distancing_unscaled,0.095765,0.224079,-0.034366,1.000000,0.018903
LogReturn,-0.043874,0.017769,-0.055334,0.018903,1.000000


In [195]:
corr

,Financial Crisis_unscaled,Depression_unscaled,Bailout_unscaled,Mortgage Crisis_unscaled,Wall Street_unscaled,Oil_unscaled,Stock Market_unscaled,Subprime_unscaled,loan_unscaled,core inflation_unscaled,...,Brent_unscaled,WTI_unscaled,metals_unscaled,inflation_unscaled,economics_unscaled,stocks_unscaled,gold_unscaled,silver_unscaled,S&P 500_unscaled,LogReturn
Financial Crisis_unscaled,1.000000,0.157636,0.011693,0.145559,0.212418,-0.087187,0.099160,-0.033465,-0.021039,-0.000171,...,-0.056785,0.030554,0.153992,0.063482,0.149618,0.117395,0.064123,0.072245,0.099035,-0.023131
Depression_unscaled,0.157636,1.000000,-0.034388,0.089934,0.095719,0.059893,-0.200762,0.081532,0.162239,0.004806,...,0.022734,0.131285,0.431580,0.457333,0.452657,0.006910,0.036586,-0.077259,-0.123286,-0.053572
Bailout_unscaled,0.011693,-0.034388,1.000000,0.049558,0.114151,-0.071240,0.057604,-0.013349,0.035180,-0.082935,...,-0.070521,-0.047481,0.035639,0.053081,-0.016105,0.150794,0.033643,0.053876,0.044592,0.010802
Mortgage Crisis_unscaled,0.145559,0.089934,0.049558,1.000000,0.060120,-0.087713,0.011250,-0.016170,0.043665,-0.035153,...,-0.021620,-0.043223,0.034870,0.085822,0.057798,0.025736,0.098272,-0.045357,0.030513,0.033776
Wall Street_unscaled,0.212418,0.095719,0.114151,0.060120,1.000000,-0.198794,0.242708,0.016020,-0.056936,0.082435,...,-0.019844,-0.095603,0.185866,0.223677,0.105627,0.331074,0.119614,0.358940,0.111953,-0.025487
Oil_unscaled,-0.087187,0.059893,-0.071240,-0.087713,-0.198794,1.000000,0.101984,-0.077769,0.444171,0.017620,...,0.344703,0.488162,-0.096169,-0.074513,-0.177856,0.192012,-0.055413,-0.061088,0.067516,-0.092357
Stock Market_unscaled,0.099160,-0.200762,0.057604,0.011250,0.242708,0.101984,1.000000,0.006524,0.034945,0.025237,...,0.154618,0.099592,-0.115710,-0.162283,-0.078371,0.657574,-0.052585,0.179625,0.605714,-0.081559
Subprime_unscaled,-0.033465,0.081532,-0.013349,-0.016170,0.016020,-0.077769,0.006524,1.000000,-0.051100,0.072969,...,0.054634,-0.034873,0.106044,0.039947,0.168409,0.033513,0.028216,-0.010983,0.055456,0.024307
loan_unscaled,-0.021039,0.162239,0.035180,0.043665,-0.056936,0.444171,0.034945,-0.051100,1.000000,-0.053546,...,0.137209,0.202034,0.009266,0.080759,0.012506,0.171429,-0.027542,-0.036319,-0.005748,-0.046759
core inflation_unscaled,-0.000171,0.004806,-0.082935,-0.035153,0.082435,0.017620,0.025237,0.072969,-0.053546,1.000000,...,0.059766,-0.023717,0.084446,0.071324,0.049641,0.034191,0.039854,-0.000419,0.120523,0.065264


In [194]:
pvals = [] 
for col in all_descriptors.columns:
    pvals.append(scipy.stats.linregress(merged['LogReturn'][1:], merged[col][1:])[3])
corr_signif = pd.DataFrame(pvals,columns = ['p-value']
                           ,index = all_descriptors.columns)
corr_signif

,p-value
Financial Crisis_unscaled,0.576279
Depression_unscaled,0.195322
Bailout_unscaled,0.794149
Mortgage Crisis_unscaled,0.414433
Wall Street_unscaled,0.538055
Oil_unscaled,0.025369
Stock Market_unscaled,0.048450
Subprime_unscaled,0.557044
loan_unscaled,0.258423
core inflation_unscaled,0.114523


In [170]:
pvals = [] 
for col in descriptors_covid.columns:
    pvals.append(scipy.stats.linregress(merged2['LogReturn'][1:], merged2[col][1:])[3])
corr_signif = pd.DataFrame(pvals,columns = ['p-value']
                           ,index = descriptors_covid.columns)
corr_signif

,p-value
covid-19_unscaled,0.599795
pandemic_unscaled,0.462062
covid vaccine_unscaled,0.469975
social distancing_unscaled,0.485958


In [196]:
def grangerTest(exog, endog):
    MAX_LAG = 30
    ARaic = AR(exog.tolist()).fit(maxlag=MAX_LAG, ic="aic")
    ARbic = AR(exog.tolist()).fit(maxlag=MAX_LAG, ic="bic")
    # select the fewer number of parameters between both criteria.
    numExog = len(ARaic.params) if len(ARaic.params) < len(ARbic.params) else len(ARbic.params)

    print ("Optimal number of lags for exog data is " + str(numExog))

    ARaic = AR(endog.tolist()).fit(maxlag=MAX_LAG, ic="aic")
    ARbic = AR(endog.tolist()).fit(maxlag=MAX_LAG, ic="bic")
    # select the fewer number of parameters between both criteria.
    numEndog = len(ARaic.params) if len(ARaic.params) < len(ARbic.params) else len(ARbic.params)

    print ("Optimal number of lags for endog data is " + str(numEndog))

    # now that I know the optimal number of parameters, I can call the
    # granger causality function of statsmodels.
    data = pd.concat([endog, exog], axis=1)
    print ("\nGranger causality results of indep onto dep")
    results = grangercausalitytests(data, maxlag=numEndog)

    data = pd.concat([exog, endog], axis=1)
    print ("\nGranger causality results of dep onto indep")
    results = grangercausalitytests(data, maxlag=numExog)
    regr = results[2][1]
    print (regr[0].params)
    print (regr[1].params)
    print (regr[1].pvalues)
for col in all_descriptors.columns:
    print('\n'+col)
    grangerTest(merged[col][1:], merged['LogReturn'][1:])


Financial Crisis_unscaled
Optimal number of lags for exog data is 4


C:\Users\Windows\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0664  , p=0.3022  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=1.0719  , p=0.3005  , df=1
likelihood ratio test: chi2=1.0709  , p=0.3007  , df=1
parameter F test:         F=1.0664  , p=0.3022  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7158  , p=0.4892  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=1.4439  , p=0.4858  , df=2
likelihood ratio test: chi2=1.4421  , p=0.4862  , df=2
parameter F test:         F=0.7158  , p=0.4892  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6142  , p=0.6060  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=1.8650  , p=0.6009  , df=3
likelihood ratio test: chi2=1.8620  , p=0.6015  , df=3
parameter F test:         F=0.6142  , p=0.6060  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.6633  , p=0.0561  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=3.6822  , p=0.0550  , df=1
likelihood ratio test: chi2=3.6707  , p=0.0554  , df=1
parameter F test:         F=3.6633  , p=0.0561  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0003  , p=0.1362  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=4.0352  , p=0.1330  , df=2
likelihood ratio test: chi2=4.0213  , p=0.1339  , df=2
parameter F test:         F=2.0003  , p=0.1362  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4730  , p=0.2208  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=4.4726  , p=0.2148  , df=3
likelihood ratio test: chi2=4.4555  , p=0.2163  , df=3
parameter F test:         F=1.4730  , p=0.2208  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2193  , p=0.6398  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=0.2204  , p=0.6387  , df=1
likelihood ratio test: chi2=0.2203  , p=0.6388  , df=1
parameter F test:         F=0.2193  , p=0.6398  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6158  , p=0.5406  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=1.2422  , p=0.5373  , df=2
likelihood ratio test: chi2=1.2409  , p=0.5377  , df=2
parameter F test:         F=0.6158  , p=0.5406  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.4969  , p=0.6846  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=1.5088  , p=0.6802  , df=3
likelihood ratio test: chi2=1.5069  , p=0.6807  , df=3
parameter F test:         F=0.4969  , p=0.6846  , df_denom=576, df_num=3

Gran

...

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.6356  , p=0.4256  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=0.6389  , p=0.4241  , df=1
likelihood ratio test: chi2=0.6385  , p=0.4242  , df=1
parameter F test:         F=0.6356  , p=0.4256  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3870  , p=0.6793  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=0.7806  , p=0.6768  , df=2
likelihood ratio test: chi2=0.7801  , p=0.6770  , df=2
parameter F test:         F=0.3870  , p=0.6793  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.4148  , p=0.7425  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=1.2594  , p=0.7388  , df=3
likelihood ratio test: chi2=1.2581  , p=0.7391  , df=3
parameter F test:         F=0.4148  , p=0.7425  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.4110  , p=0.5217  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=0.4131  , p=0.5204  , df=1
likelihood ratio test: chi2=0.4130  , p=0.5205  , df=1
parameter F test:         F=0.4110  , p=0.5217  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.2543  , p=0.7756  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=0.5130  , p=0.7738  , df=2
likelihood ratio test: chi2=0.5127  , p=0.7739  , df=2
parameter F test:         F=0.2543  , p=0.7756  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.2689  , p=0.8478  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=0.8166  , p=0.8455  , df=3
likelihood ratio test: chi2=0.8160  , p=0.8456  , df=3
parameter F test:         F=0.2689  , p=0.8478  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0566  , p=0.8120  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=0.0569  , p=0.8114  , df=1
likelihood ratio test: chi2=0.0569  , p=0.8114  , df=1
parameter F test:         F=0.0566  , p=0.8120  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8778  , p=0.4163  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=1.7707  , p=0.4126  , df=2
likelihood ratio test: chi2=1.7680  , p=0.4131  , df=2
parameter F test:         F=0.8778  , p=0.4163  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.5249  , p=0.6653  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=1.5938  , p=0.6608  , df=3
likelihood ratio test: chi2=1.5916  , p=0.6613  , df=3
parameter F test:         F=0.5249  , p=0.6653  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.1480  , p=0.2844  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=1.1539  , p=0.2827  , df=1
likelihood ratio test: chi2=1.1527  , p=0.2830  , df=1
parameter F test:         F=1.1480  , p=0.2844  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5389  , p=0.5837  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=1.0870  , p=0.5807  , df=2
likelihood ratio test: chi2=1.0860  , p=0.5810  , df=2
parameter F test:         F=0.5389  , p=0.5837  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4382  , p=0.2306  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=4.3669  , p=0.2245  , df=3
likelihood ratio test: chi2=4.3506  , p=0.2260  , df=3
parameter F test:         F=1.4382  , p=0.2306  , df_denom=576, df_num=3

Gran

Optimal number of lags for endog data is 10

Granger causality results of indep onto dep

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0360  , p=0.8495  , df_denom=582, df_num=1
ssr based chi2 test:   chi2=0.0362  , p=0.8491  , df=1
likelihood ratio test: chi2=0.0362  , p=0.8491  , df=1
parameter F test:         F=0.0360  , p=0.8495  , df_denom=582, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6469  , p=0.5240  , df_denom=579, df_num=2
ssr based chi2 test:   chi2=1.3050  , p=0.5207  , df=2
likelihood ratio test: chi2=1.3036  , p=0.5211  , df=2
parameter F test:         F=0.6469  , p=0.5240  , df_denom=579, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.4780  , p=0.6977  , df_denom=576, df_num=3
ssr based chi2 test:   chi2=1.4514  , p=0.6935  , df=3
likelihood ratio test: chi2=1.4496  , p=0.6940  , df=3
parameter F test:         F=0.4780  , p=0.6977  , df_denom=576, df_num=3

Gran

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(descriptors[['Oil_unscaled','Brent_unscaled','WTI_unscaled','metals_unscaled','stocks_unscaled']])
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2'])
principalDf.head()

In [ ]:
principalDf.index = descriptors.index
principalDf.plot()

In [ ]:
merged = pd.merge(merged,principalDf, how= 'inner', left_index=True, right_index=True)
merged.head()

In [ ]:
grangerTest(merged['PC1'][1:], merged['LogReturn'][1:])

In [ ]:
descriptors.to_csv('Google_Trends_variables_1stPart_2005_2015.tsv', sep='\t')

In [119]:
all_descriptors.to_csv('Google_Trends_variables_all.tsv', sep='\t')
descriptors_covid.to_csv('Google_Trends_variables_covid.tsv', sep='\t')

In [201]:
all_descriptors.to_csv('Google_Trends_variables_all.tsv', sep='\t')

In [ ]:
dailydata.get_daily_data(word, 2005, 1, 2021, 5, geo = 'US')
not significant